In [1]:
from parsing import parser
from parsing_ast import transformer
import parsing_ast

In [5]:
input_str = """ 
var a = true;
if(a) {

}
"""

In [6]:
tree = parser.parse(input_str)
ast = transformer.transform(tree)

DEBUG:parsing_ast:IDENTIFIER - a
DEBUG:parsing_ast:literal - [Token('BOOLEAN_VALUE', 'true')]
DEBUG:parsing_ast:expression - [literal('true')]
DEBUG:parsing_ast:Processing Variable Declaration with [declaration_variable('var'), identifier('a'), Token('ASSIGNMENT_OPERATOR', '='), literal('true'), Token('END_OF_STATEMENT', ';')]
DEBUG:parsing_ast:Variable Declaration - var, [identifier('a')], [literal('true')]
DEBUG:parsing_ast:Processing 'declare' with [declaration(declaration_variable('var'), [identifier('a')], [literal('true')])]
DEBUG:parsing_ast:Processing 'statement' with [declaration(declaration_variable('var'), [identifier('a')], [literal('true')])]
DEBUG:parsing_ast:IDENTIFIER - a
DEBUG:parsing_ast:expression - [identifier('a')]
DEBUG:parsing_ast:Processing Block with [Token('CURLY_OPEN', '{'), Token('CURLY_CLOSE', '}')]
DEBUG:parsing_ast:Processing Else Temp with []
DEBUG:parsing_ast:Processing Control with [if_else(identifier('a'), block(None), None)]
DEBUG:parsing_ast:Process

['var', 'a', '=', 'true', ';', 'if', '(', 'a', ')', '{', '}']


In [7]:
ast

statements([declaration(declaration_variable('var'), [identifier('a')], [literal('true')]), if_else(identifier('a'), block(None), None)])

In [8]:
class Symbol(object):
    def __init__(self, name, type=None, category=None) -> None:
        self.name = name
        self.type = type
        self.category = category

In [9]:
class BuiltinTypeSymbol(Symbol):
    def __init__(self, name) -> None:
        super().__init__(name)

    def __str__(self) -> str:
        return self.name
    
    def __repr__(self) -> str:
        return "<{class_name}(name='{name}')>".format(
            class_name=self.__class__.__name__,
            name=self.name,
        )

In [10]:
class VarSymbol(Symbol):
    def __init__(self, name, type=None, category=None) -> None:
        super().__init__(name, type, category)
    
    def __str__(self) -> str:
        return "<{class_name}(name='{name}', type='{type}', category='{category}')>".format(
            class_name=self.__class__.__name__,
            name=self.name,
            type=self.type,
            category=self.category
        )
    
    __repr__ = __str__

In [13]:
class SymbolTable(object):
    def __init__(self) -> None:
        self._symbols = {}
        self._init_builtins()

    def _init_builtins(self) -> None:
        self.insert(BuiltinTypeSymbol('INTEGER_CONSTANT'))
        self.insert(BuiltinTypeSymbol('DECIMAL_CONSTANT'))
        self.insert(BuiltinTypeSymbol('STRING_LITERAL'))
        self.insert(BuiltinTypeSymbol('BOOLEAN_VALUE'))
        self.insert(BuiltinTypeSymbol('NULL_VALUE'))
        self.insert(BuiltinTypeSymbol('ARRAY'))
        self.insert(BuiltinTypeSymbol('TUPLE'))
        self.insert(BuiltinTypeSymbol('LIST'))

    def __str__(self) -> str:
        symtab_header = 'Symbol table contents'
        lines = ['\n', symtab_header, '_' * len(symtab_header)]
        lines.extend(
            ('%7s: %r' % (key, value))
            for key, value in self._symbols.items()
        )
        lines.append('\n')
        s = '\n'.join(lines)
        return s
    
    __repr__ = __str__

    def insert(self, symbol) -> None:
        print('Insert: %s' % symbol.name)
        self._symbols[symbol.name] = symbol

    def lookup(self, name) -> Symbol:
        print('Lookup: %s' % name)
        return self._symbols.get(name, None)

In [175]:
input_str = """ 
var a = 1;
a = a++;
"""
tree = parser.parse(input_str)
ast = transformer.transform(tree)

DEBUG:parsing_ast:IDENTIFIER - a
DEBUG:parsing_ast:literal - [Token('INTEGER_CONSTANT', '1')]
DEBUG:parsing_ast:expression - [literal('1')]
DEBUG:parsing_ast:Processing Variable Declaration with [declaration_variable('var'), identifier('a'), Token('ASSIGNMENT_OPERATOR', '='), literal('1'), Token('END_OF_STATEMENT', ';')]
DEBUG:parsing_ast:Variable Declaration - var, [identifier('a')], [literal('1')]
DEBUG:parsing_ast:Processing 'declare' with [declaration(declaration_variable('var'), [identifier('a')], [literal('1')])]
DEBUG:parsing_ast:Processing 'statement' with [declaration(declaration_variable('var'), [identifier('a')], [literal('1')])]
DEBUG:parsing_ast:IDENTIFIER - a
DEBUG:parsing_ast:IDENTIFIER - a
DEBUG:parsing_ast:unary_expression - [identifier('a'), unary_operator('++')]
DEBUG:parsing_ast:expression - [unary_expression(identifier('a'), unary_operator('++'))]
DEBUG:parsing_ast:expression_statement - [assignment(identifier('a'), unary_expression(identifier('a'), unary_operator(

['var', 'a', '=', '1', ';', 'a', '=', 'a', '++', ';']


In [176]:
symbol_table = SymbolTable()
symbol_stack = [symbol_table]

Insert: INTEGER_CONSTANT
Insert: DECIMAL_CONSTANT
Insert: STRING_LITERAL
Insert: BOOLEAN_VALUE
Insert: NULL_VALUE
Insert: ARRAY
Insert: TUPLE
Insert: LIST


In [177]:
def deduce_type(node, symbol_table):
    if isinstance(node, parsing_ast.BinaryOp):
        left_type = deduce_type(node.children[0], symbol_table)
        right_type = deduce_type(node.children[1], symbol_table)
        if isinstance(node.operator, parsing_ast.Operator):
            if node.operator.operator in ['+','+=']:
                if left_type.name == 'STRING_LITERAL' and right_type.name == 'STRING_LITERAL':
                    return symbol_table.lookup('STRING_LITERAL')
                if left_type.name == 'INTEGER_CONSTANT' and right_type.name == 'INTEGER_CONSTANT':
                    return symbol_table.lookup('INTEGER_CONSTANT')
                if left_type.name == 'DECIMAL_CONSTANT' and right_type.name == 'DECIMAL_CONSTANT':
                    return symbol_table.lookup('DECIMAL_CONSTANT')
                if left_type.name == 'INTEGER_CONSTANT' and right_type.name == 'DECIMAL_CONSTANT':
                    return symbol_table.lookup('DECIMAL_CONSTANT')
                if left_type.name == 'DECIMAL_CONSTANT' and right_type.name == 'INTEGER_CONSTANT':
                    return symbol_table.lookup('DECIMAL_CONSTANT')
                raise Exception('Type mismatch')
            elif node.operator.operator in ['-', '*', '-=', '*=']:
                if left_type.name == 'INTEGER_CONSTANT' and right_type.name == 'INTEGER_CONSTANT':
                    return symbol_table.lookup('INTEGER_CONSTANT')
                if left_type.name == 'DECIMAL_CONSTANT' and right_type.name == 'DECIMAL_CONSTANT':
                    return symbol_table.lookup('DECIMAL_CONSTANT')
                if left_type.name == 'INTEGER_CONSTANT' and right_type.name == 'DECIMAL_CONSTANT':
                    return symbol_table.lookup('DECIMAL_CONSTANT')
                if left_type.name == 'DECIMAL_CONSTANT' and right_type.name == 'INTEGER_CONSTANT':
                    return symbol_table.lookup('DECIMAL_CONSTANT')
                raise Exception('Type mismatch')
            elif node.operator.operator in ['/','%','/=', '%=']:
                if left_type.name == 'INTEGER_CONSTANT' and right_type.name == 'INTEGER_CONSTANT':
                    return symbol_table.lookup('DECIMAL_CONSTANT')
                if left_type.name == 'DECIMAL_CONSTANT' and right_type.name == 'DECIMAL_CONSTANT':
                    return symbol_table.lookup('DECIMAL_CONSTANT')
                if left_type.name == 'INTEGER_CONSTANT' and right_type.name == 'DECIMAL_CONSTANT':
                    return symbol_table.lookup('DECIMAL_CONSTANT')
                if left_type.name == 'DECIMAL_CONSTANT' and right_type.name == 'INTEGER_CONSTANT':
                    return symbol_table.lookup('DECIMAL_CONSTANT')
                raise Exception('Type mismatch')
            elif node.operator.operator in ['&', '|']:
                if left_type.name == 'BOOLEAN_VALUE' and right_type.name == 'BOOLEAN_VALUE':
                    return symbol_table.lookup('BOOLEAN_VALUE')
                if left_type.name == 'INTEGER_CONSTANT' and right_type.name == 'INTEGER_CONSTANT':
                    return symbol_table.lookup('INTEGER_CONSTANT')
                if left_type.name == 'DECIMAL_CONSTANT' and right_type.name == 'DECIMAL_CONSTANT':
                    return symbol_table.lookup('DECIMAL_CONSTANT')
                if left_type.name == 'INTEGER_CONSTANT' and right_type.name == 'DECIMAL_CONSTANT':
                    return symbol_table.lookup('DECIMAL_CONSTANT')
                if left_type.name == 'DECIMAL_CONSTANT' and right_type.name == 'INTEGER_CONSTANT':
                    return symbol_table.lookup('DECIMAL_CONSTANT')
                raise Exception('Type mismatch')
            elif node.operator.operator in ["<<=", ">>=", "&=", "|=", "^="]:
                if left_type == right_type:
                    return symbol_table.lookup('BOOLEAN_VALUE')
                raise Exception('Type mismatch')
        elif isinstance(node.operator, parsing_ast.Comparator) and node.operator.comparator in ['==', '!=', '<=', '>=', '<', '>', '||', '&&']:
            if left_type == right_type:
                return symbol_table.lookup('BOOLEAN_VALUE')
            raise Exception('Type mismatch')
        else:
            raise Exception('Unknown operator')    
    if isinstance(node, parsing_ast.Literal):
        return symbol_table.lookup(node.type)
    if isinstance(node, parsing_ast.Identifier):
        if not symbol_table.lookup(node.name):
            raise Exception('Error: Symbol not found: %s' %node.name)

In [178]:
def traverse_ast(node, symbol_table, last=True):
    if isinstance(node, list):
        for idx, item in enumerate(node):
            if idx == len(node) - 1:
                traverse_ast(item, last=True, symbol_table=symbol_table)
            else:
                traverse_ast(item, last=False, symbol_table=symbol_table)
        return
    print(type(node))
    if type(node) == parsing_ast.Function:
        if symbol_table.lookup(node.name.name):
            raise Exception('Error: Duplicate Function Declaration: %s' % node.name.name)
        symbol_table.insert(VarSymbol(node.name.name, 'function'))
        new_table = SymbolTable()
        symbol_stack.append(new_table)
        for args in node.parameters.parameters:
            if new_table.lookup(args.identifier.name):
                raise Exception('Error: Duplicate Function Argument: %s' % args.identifier.name)
            new_table.insert(VarSymbol(args.identifier.name, args.declaration_type.value))
        children = [(k, v) for k, v in node.__dict__.items() if not k.startswith('_') and v]
        for idx, (attr, child) in enumerate(children):
            is_last = idx == len(children) - 1
            traverse_ast(child, last=is_last, symbol_table=new_table)
        return
    if isinstance(node, parsing_ast.InbuiltFunctionCall):
        if not symbol_table.lookup(node.base.name):
            raise Exception('Error: Symbol not found: %s' % node.base.name)
    if type(node) == parsing_ast.FunctionCall:
        if not symbol_table.lookup(node.name.name):
            raise Exception('Error: Symbol not found: %s' % node.name.name)
    if isinstance(node, parsing_ast.VariableDeclaration):
        if len(node.identifier) != len(node.value):
            raise Exception('Error: Mismatch in number of identifiers and values')
        for idx, identifier in enumerate(node.identifier):
            if symbol_table.lookup(identifier.name):
                raise Exception('Error: Duplicate identifier found: %s' % identifier.name)
            if isinstance(node.value[idx], parsing_ast.Identifier) and not symbol_table.lookup(node.value[idx].name):
                raise Exception('Error: Symbol not found: %s' % node.value[idx].name)
            if isinstance(node.value[idx], parsing_ast.Identifier):
                symbol_table.insert(VarSymbol(identifier.name, symbol_table.lookup(node.value[idx].name).type, node.declaration_type.value))
            if isinstance(node.value[idx], parsing_ast.Literal):
                symbol_table.insert(VarSymbol(identifier.name, symbol_table.lookup(node.value[idx].type), node.declaration_type.value))
            if isinstance(node.value[idx], parsing_ast.BinaryOp):
                symbol_table.insert(VarSymbol(identifier.name, deduce_type(node.value[idx], symbol_table), node.declaration_type.value))
    if isinstance(node, parsing_ast.Declaration):
        if symbol_table.lookup(node.identifier.name):
            raise Exception('Error: Duplicate identifier found: %s' % node.identifier.name)
        if node.declaration_type.value == 'tuple':
            symbol_table.insert(VarSymbol(node.identifier.name, symbol_table.lookup('TUPLE'), node.declaration_type.value))
        if node.declaration_type.value == 'list':
            symbol_table.insert(VarSymbol(node.identifier.name, symbol_table.lookup('LIST'), node.declaration_type.value))
        if node.declaration_type.value == 'array':
            symbol_table.insert(VarSymbol(node.identifier.name, symbol_table.lookup('ARRAY'), node.declaration_type.value))
        if isinstance(node.value, parsing_ast.Identifier) and not symbol_table.lookup(node.value.name):
            raise Exception('Error: Symbol not found: %s' % node.value.name)
        if isinstance(node.value, parsing_ast.Identifier):
            symbol_table.insert(VarSymbol(node.identifier.name, symbol_table.lookup(node.value.name).type, node.declaration_type.value))
        if isinstance(node.value, parsing_ast.Literal):
            symbol_table.insert(VarSymbol(node.identifier.name, symbol_table.lookup(node.value.type), node.declaration_type.value))
        if isinstance(node.value, parsing_ast.BinaryOp):
            symbol_table.insert(VarSymbol(node.identifier.name, deduce_type(node.expression, symbol_table), node.declaration_type.value))
    if isinstance(node, parsing_ast.Assignment):
        if not symbol_table.lookup(node.identifier.name):
            raise Exception('Error: Symbol not found: %s' % node.identifier.name)
        if isinstance(node.expression, parsing_ast.Identifier) and not symbol_table.lookup(node.expression.name):
            raise Exception('Error: Symbol not found: %s' % node.expression.name)
        if isinstance(node.expression, parsing_ast.Identifier):
            symbol = symbol_table.lookup(node.identifier.name)
            symbol.type = symbol_table.lookup(node.expression.name).type
        if isinstance(node.expression, parsing_ast.Literal):
            symbol = symbol_table.lookup(node.identifier.name)
            symbol.type = symbol_table.lookup(node.expression.type)
        if isinstance(node.expression, parsing_ast.BinaryOp):
            symbol = symbol_table.lookup(node.identifier.name)
            symbol.type = deduce_type(node.expression, symbol_table)
    if type(node) == parsing_ast.FunctionArgumentList:
        for arg in node.arguments:
            if isinstance(arg, parsing_ast.Identifier) and not symbol_table.lookup(arg.name):
                raise Exception('Error: Symbol not found: %s' % arg.name)
    if type(node) == parsing_ast.PrintArgumentList:
        for arg in node.arguments:
            if isinstance(arg, parsing_ast.Identifier) and not symbol_table.lookup(arg.name):
                raise Exception('Error: Symbol not found: %s' % arg.name)
    if type(node) == parsing_ast.IfElse:
        if isinstance(node.expression, parsing_ast.Identifier) and not symbol_table.lookup(node.expression.name):
            raise Exception('Error: Symbol not found: %s' % node.expression.name)
    if type(node) == parsing_ast.While:
        if isinstance(node.expression, parsing_ast.Identifier) and not symbol_table.lookup(node.expression.name):
            raise Exception('Error: Symbol not found: %s' % node.expression.name)
    if type(node) == parsing_ast.DoWhile:
        if isinstance(node.expression, parsing_ast.Identifier) and not symbol_table.lookup(node.expression.name):
            raise Exception('Error: Symbol not found: %s' % node.expression.name)
    if type(node) == parsing_ast.BinaryOp:
        if isinstance(node.children[0], parsing_ast.Identifier) and not symbol_table.lookup(node.children[0].name):
            raise Exception('Error: Symbol not found: %s' % node.children[0].name)
        if isinstance(node.children[1], parsing_ast.Identifier) and not symbol_table.lookup(node.children[1].name):
            raise Exception('Error: Symbol not found: %s' % node.children[1].name)
    if type(node) == parsing_ast.UnaryExpression:
        if isinstance(node.children[0], parsing_ast.Identifier) and not symbol_table.lookup(node.children[0].name):
            raise Exception('Error: Symbol not found: %s' % node.children[0].name)
        if isinstance(node.operator, parsing_ast.Identifier) and not symbol_table.lookup(node.operator.name):
            raise Exception('Error: Symbol not found: %s' % node.operator.name)
    if not hasattr(node, '__dict__'):
        return 
    children = [(k, v) for k, v in node.__dict__.items() if not k.startswith('_') and v]
    for idx, (attr, child) in enumerate(children):
        is_last = idx == len(children) - 1
        traverse_ast(child, last=is_last, symbol_table=symbol_table)

In [179]:
traverse_ast(ast, symbol_table)

<class 'parsing_ast.Statements'>
<class 'parsing_ast.VariableDeclaration'>
Lookup: a
Lookup: INTEGER_CONSTANT
Insert: a
<class 'parsing_ast.DeclarationVariable'>
<class 'str'>
<class 'parsing_ast.Identifier'>
<class 'str'>
<class 'parsing_ast.Literal'>
<class 'str'>
<class 'str'>
<class 'parsing_ast.Assignment'>
Lookup: a
<class 'parsing_ast.Identifier'>
<class 'str'>
<class 'parsing_ast.UnaryExpression'>
Lookup: a
<class 'parsing_ast.UnaryOperator'>
<class 'str'>
<class 'parsing_ast.Identifier'>
<class 'str'>


In [173]:
symbol = symbol_table.lookup('INTEGER_CONSTANT')
symbol.name

Lookup: INTEGER_CONSTANT


'INTEGER_CONSTANT'

In [161]:
symbol = symbol_table.lookup('a')

Lookup: a


In [180]:
symbol_stack

[
 
 Symbol table contents
 _____________________
 INTEGER_CONSTANT: <BuiltinTypeSymbol(name='INTEGER_CONSTANT')>
 DECIMAL_CONSTANT: <BuiltinTypeSymbol(name='DECIMAL_CONSTANT')>
 STRING_LITERAL: <BuiltinTypeSymbol(name='STRING_LITERAL')>
 BOOLEAN_VALUE: <BuiltinTypeSymbol(name='BOOLEAN_VALUE')>
 NULL_VALUE: <BuiltinTypeSymbol(name='NULL_VALUE')>
   ARRAY: <BuiltinTypeSymbol(name='ARRAY')>
   TUPLE: <BuiltinTypeSymbol(name='TUPLE')>
    LIST: <BuiltinTypeSymbol(name='LIST')>
       a: <VarSymbol(name='a', type='INTEGER_CONSTANT', category='var')>
 ]

In [74]:
a = 1
type(a)

int

In [75]:
b = 1.0
type(b)

float

In [77]:
b = a
type(b)

int